# These steps are to run before running KITTI RAW data on the yolo-slam 

##  1. Calculating the calibration matrix

the calibration matrix is used for filling in the camera intrisic parameters
in settings files (.yaml) under ORB_SLAM repository

In [2]:
import pandas as pd
import numpy as np

In [6]:
path = '/home/brwei01/Data/data_tracking/data_tracking_calibration/training/calib/0000.txt'
calibration = pd.read_csv(path, delimiter=' ', header=None, index_col=0)
columns_to_drop = [13, 14]
calibration = calibration.drop(columns=columns_to_drop)
calibration

,1,2,3,4,5,6,7,8,9,10,11,12
0,,,,,,,,,,,,
P0:,721.537700,0.000000,609.559300,0.000000,0.000000,721.537700,172.854000,0.000000,0.000000,0.000000,1.000000,0.000000
P1:,721.537700,0.000000,609.559300,-387.574400,0.000000,721.537700,172.854000,0.000000,0.000000,0.000000,1.000000,0.000000
P2:,721.537700,0.000000,609.559300,44.857280,0.000000,721.537700,172.854000,0.216379,0.000000,0.000000,1.000000,0.002746
P3:,721.537700,0.000000,609.559300,-339.524200,0.000000,721.537700,172.854000,2.199936,0.000000,0.000000,1.000000,0.002730
R_rect,0.999924,0.009838,-0.007445,-0.009870,0.999942,-0.004278,0.007403,0.004352,0.999963,NaN,NaN,NaN
Tr_velo_cam,0.007534,-0.999971,-0.000617,-0.004070,0.014802,0.000728,-0.999890,-0.076316,0.999862,0.007524,0.014808,-0.271781
Tr_imu_velo,0.999998,0.000755,-0.002036,-0.808676,-0.000785,0.999890,-0.014823,0.319556,0.002024,0.014825,0.999888,-0.799723


P0,P1,P2,P3 are the camera projection matrices where P0,P1 are for the two grayscale cameras and the remaining two for RGB cameras. Tr is a translation vector that transforms points from velodyne sensor coordinates to respective rectified camera coordinate systems.



In [7]:
P0 = np.array(calibration.iloc[0]).reshape((3,4))
P0

array([[721.5377,   0.    , 609.5593,   0.    ],
       [  0.    , 721.5377, 172.854 ,   0.    ],
       [  0.    ,   0.    ,   1.    ,   0.    ]])

In [8]:
P1 = np.array(calibration.iloc[1]).reshape((3,4))
P1

array([[ 721.5377,    0.    ,  609.5593, -387.5744],
       [   0.    ,  721.5377,  172.854 ,    0.    ],
       [   0.    ,    0.    ,    1.    ,    0.    ]])

In [10]:
P2 = np.array(calibration.iloc[2]).reshape((3,4))
P2

array([[7.215377e+02, 0.000000e+00, 6.095593e+02, 4.485728e+01],
       [0.000000e+00, 7.215377e+02, 1.728540e+02, 2.163791e-01],
       [0.000000e+00, 0.000000e+00, 1.000000e+00, 2.745884e-03]])

In [11]:
P3 = np.array(calibration.iloc[3]).reshape((3,4))
P3

array([[ 7.215377e+02,  0.000000e+00,  6.095593e+02, -3.395242e+02],
       [ 0.000000e+00,  7.215377e+02,  1.728540e+02,  2.199936e+00],
       [ 0.000000e+00,  0.000000e+00,  1.000000e+00,  2.729905e-03]])

## 2. Renaming the file names of frames (raw) to the correct format (%06d)%

Do this step so that this could be read by yolo-slam system. The example files in ORB-SLAM2 was designe for KITTI odometry dataset, where frames are named with (%06d)% format

In [ ]:
import os

# Specify the folder path
folder_path = '/home/brwei01/Data/data_tracking/sequences/000000/image_0'

# Get a list of all files in the folder
file_list = os.listdir(folder_path)

# Sort the file list to ensure correct ordering
file_list.sort()

# Iterate through the files and rename them
for index, filename in enumerate(file_list):
    old_filepath = os.path.join(folder_path, filename)
    new_filename = f"{index:06d}.png"  # Formats the index as 6-digit zero-padded number
    new_filepath = os.path.join(folder_path, new_filename)
    
    os.rename(old_filepath, new_filepath)
    
print("Files renamed successfully.")

## 3.Converting timestamps of raw data to the format same as VO data

in kitti raw data, the time stamps are:
2011-09-26 13:04:32.345808896
2011-09-26 13:04:32.449188864
2011-09-26 13:04:32.552435968

in VO data, the time stamps are:
0.000000e+00
1.033914e-01
2.071056e-01
3.102615e-01

In [6]:
from datetime import datetime
import numpy as np

In [7]:
input_path = '/home/brwei01/Data/data_tracking/sequences/000000/timestamps.txt' 
output_path = '/home/brwei01/Data/data_tracking/sequences/000000/times.txt' 

In [ ]:
timestamp_strs = []
with open(input_path) as f:
    for line in f:
        timestamp_strs.append(line)

# Convert timestamp strings to datetime objects
#  ts[:26] to truncate the timestamp strings to the first 26 characters, which correspond to the format "YYYY-MM-DD HH:MM:SS."
timestamps = [datetime.strptime(ts[:26], "%Y-%m-%d %H:%M:%S.%f") for ts in timestamp_strs]
# Calculate time differences relative to the start time
time_deltas = [(ts - timestamps[0]).total_seconds() for ts in timestamps]
# Convert time differences to scientific notation
time_series = [format(td, ".6e") for td in time_deltas]

# Print the resulting time series
with open(output_path, 'w') as f:
    for value in time_series:
        f.write(value + '\n')

## 4. Create Ground Truth poses for KITTI Raw sequences

Run this script: to calculate scale factor and make evaluations using evo package

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import sys 
import os
sys.path.append('../src/')

from data_utils import *

In [2]:
# Replace this with the actual path of your folder:
DATA_FOLDER_PATH = '/home/brwei01/Data/data_raw/2011_09_26/2011_09_26_drive_0005_sync/oxts/data/' 

# Replace this with the actual times.txt path after processing in previous step 3
TIMES_FILE_PATH = '/home/brwei01/Data/data_tracking/sequences/000000/times.txt' 

# The columns names are referenced from: https://github.com/bostondiditeam/kitti/blob/master/resources/devkit_object/readme.txt
IMU_COLUMN_NAMES = ['lat','lon','alt','roll','pitch', 'yaw','vn','ve','vf','vl','vu','ax','ay','az','af',
                    'al','au','wx','wy','wz','wf','wl','wu','posacc','velacc','navstat','numsats','posmode','velmode','orimode']

In [3]:
def euler_to_quaternion(yaw, pitch, roll):
    cy = np.cos(yaw * 0.5)
    sy = np.sin(yaw * 0.5)
    cp = np.cos(pitch * 0.5)
    sp = np.sin(pitch * 0.5)
    cr = np.cos(roll * 0.5)
    sr = np.sin(roll * 0.5)

    q1 = cy * cp * cr + sy * sp * sr
    q2 = cy * cp * sr - sy * sp * cr
    q3 = sy * cp * sr + cy * sp * cr
    q4 = sy * cp * cr - cy * sp * sr

    return q1, q2, q3, q4

def read_times(times_file_path):
    times = []
    with open(times_file_path) as f:
        for time in f:
            times.append(time)
    return times

In [4]:
file_count = 0
for filename in os.listdir(DATA_FOLDER_PATH):
    if os.path.isfile(os.path.join(DATA_FOLDER_PATH, filename)):
        file_count += 1

prev_imu_data = None
poses = []
fps = 10


for frame in range(file_count):
    imu_data = read_imu(os.path.join(DATA_FOLDER_PATH, '%010d.txt' % frame))
    times =  read_times(os.path.join(TIMES_FILE_PATH))
    if prev_imu_data is not None:
        displacement = 0.1 * np.linalg.norm(imu_data[['vf', 'vl', 'vu']])
        yaw_change = imu_data.yaw - prev_imu_data.yaw
        pitch_change = imu_data.pitch - prev_imu_data.pitch
        roll_change = imu_data.roll - prev_imu_data.roll

        for i in range(len(poses)):
            time, x0, y0, z0, q1, q2, q3, q4 = poses[i]
            
            # Calculate new x, y, and z coordinates based on yaw_change, pitch_change, and vf, vl, vu
            x1 = x0 * np.cos(yaw_change) + y0 * np.sin(yaw_change) - displacement * np.cos(pitch_change)
            y1 = -x0 * np.sin(yaw_change) + y0 * np.cos(yaw_change) - displacement * np.sin(pitch_change)
            z1 = z0 - imu_data.vu * 0.1 * fps  # Consider vertical velocity and time interval
            
            q1_new, q2_new, q3_new, q4_new = euler_to_quaternion(yaw_change, pitch_change, roll_change)

            time_curr = pd.Series(times[i].strip())

            poses[i] = np.array([time_curr, x1, y1, z1, q1 * q1_new, q2 * q2_new, q3 * q3_new, q4 * q4_new])
    
    poses.append(np.array([0, 0, 0, 0, 1, 0, 0, 0]))
    prev_imu_data = imu_data


In [5]:
poses

[array([['0.000000e+00'],
        [-58.657392256346164],
        [-22.478254321165693],
        [7.5276313521748826],
        [0.9942802687349956],
        [0.0],
        [0.0],
        [-0.0]], dtype=object),
 array([['1.033800e-01'],
        [-58.31908225463388],
        [-22.392911012455478],
        [7.538648905478434],
        [0.9943137756403543],
        [0.0],
        [0.0],
        [-0.0]], dtype=object),
 array([['2.066270e-01'],
        [-57.984291587163106],
        [-22.301785384958695],
        [7.626079722598415],
        [0.9943468110725212],
        [0.0],
        [0.0],
        [-0.0]], dtype=object),
 array([['3.100090e-01'],
        [-57.656235015658346],
        [-22.208983085394454],
        [7.699542112785885],
        [0.9943753305301858],
        [-0.0],
        [0.0],
        [-0.0]], dtype=object),
 array([['4.132760e-01'],
        [-57.33485840914933],
        [-22.115869021161473],
        [7.698580011253065],
        [0.9943995729915875],
        [0.0],
  

In [113]:
output_path = '/home/brwei01/Data/data_tracking/sequences/000000/gt_000000.txt'
with open(output_path, 'w') as f:
    for pose in poses:
        timestamp = str(pose[0]).strip('[]').strip("' '")
        x = str(pose[1]).strip('[]')
        y = str(pose[2]).strip('[]')
        z = str(pose[3]).strip('[]')
        q1 = str(pose[4]).strip('[]')
        q2 = str(pose[5]).strip('[]')
        q3 = str(pose[6]).strip('[]')
        q4 = str(pose[7]).strip('[]')
        line = f'{timestamp} {x} {y} {z} {q1} {q2} {q3} {q4}\n'
        f.write(line)
        

# These steps are to run AFTER running KITTI RAW data on the yolo-slam 

## 1. Formatting console log(detection box and distance) from SLAM
the 6 entries are : 'frame_number', 'bbox_left', 'bbox_top', 'bbox_right', 'bbox_bottom', 'distance_SLAM'

In [16]:
import re
import numpy as np

In [17]:
# setting file paths

input_path = '/home/brwei01/Dev/COMP0130_22-23_Topic_03/Coursework_03/Results/console_log.txt' 
output_path = '/home/brwei01/Dev/COMP0130_22-23_Topic_03/Coursework_03/Results/console_log_formatted.txt' 

In [18]:
# Initialize a list to store the extracted information
result = []
output = []
with open(input_path, 'r') as input_file:
    
    # Initialize variables to store information for the current frame
    frame_number = None
    bounding_box = None
    min_distance = None
    
    # Define a regular expression pattern to extract bounding box values
    bbox_pattern = r'\[(\d+), (\d+)\]\[(\d+), (\d+)\]'
    
    # Iterate through the lines and extract information
    for line in input_file:
        if line.startswith("Processing Image NO."):
            # Extract frame number
            frame_number = line.split(":")[-1].strip()
        elif line.startswith("This is bounding box:"):
            # Extract bounding box values using regex
            match = re.search(bbox_pattern, line)
            if match:
                left, top, right, bottom = map(int, match.groups())
                # Store bounding box coordinates as a list
                bounding_box = [left, top, right, bottom]
        elif line.startswith("min distance to camera:"):
            # Extract minimum distance
            min_distance = float(line.split(":")[-1])
            # Append the extracted information to the result list
            result.append({
                "frame_number": frame_number,
                "bounding_box": bounding_box,
                "distance": min_distance
            })
    
    # Print the result (list of dictionaries)
    for entry in result:
        # print(entry)
        frame_number = entry['frame_number']
        bounding_box = entry['bounding_box']
        distance = entry['distance']
        record = frame_number + ' ' + str(bounding_box[0]) + ' ' + str(bounding_box[1]) + ' ' + str(bounding_box[2]) + ' ' + str(bounding_box[3]) + ' ' + str(distance) + '\n'
        output.append(record)    


In [19]:
output

['000015 466 199 516 238 4.88335\n',
 '000015 224 187 260 211 4.88335\n',
 '000015 453 165 512 206 4.88335\n',
 '000015 240 188 271 209 4.88335\n',
 '000015 183 188 231 214 4.88335\n',
 '000015 277 160 417 259 4.88335\n',
 '000015 1203 208 1242 254 4.88335\n',
 '000015 688 187 741 228 4.88335\n',
 '000015 794 248 907 372 4.88335\n',
 '000015 961 180 1075 249 4.88335\n',
 '000016 460 166 519 203 15.7955\n',
 '000016 256 186 282 208 15.7955\n',
 '000016 482 200 524 238 15.7955\n',
 '000016 187 187 233 214 15.7955\n',
 '000016 701 187 756 229 15.7955\n',
 '000016 794 250 905 371 15.7955\n',
 '000016 981 184 1098 251 15.7955\n',
 '000016 460 166 519 203 15.7955\n',
 '000016 256 186 282 208 15.7955\n',
 '000016 482 200 524 238 15.7955\n',
 '000016 187 187 233 214 15.7955\n',
 '000016 701 187 756 229 15.7955\n',
 '000016 794 250 905 371 15.7955\n',
 '000016 981 184 1098 251 15.7955\n',
 '000017 253 183 286 206 12.1572\n',
 '000017 468 166 524 200 12.1572\n',
 '000017 484 199 535 237 12.1572\

In [20]:
# delete multiplication
unique_records = {}

# Process the input data
for record in output:
    if record not in unique_records:
        unique_records[record] = True

# Convert the dictionary keys back to a list
unique_records_list = list(unique_records.keys())

# Print the unique records
for record in unique_records_list:
    print(record)

000015 466 199 516 238 4.88335

000015 224 187 260 211 4.88335

000015 453 165 512 206 4.88335

000015 240 188 271 209 4.88335

000015 183 188 231 214 4.88335

000015 277 160 417 259 4.88335

000015 1203 208 1242 254 4.88335

000015 688 187 741 228 4.88335

000015 794 248 907 372 4.88335

000015 961 180 1075 249 4.88335

000016 460 166 519 203 15.7955

000016 256 186 282 208 15.7955

000016 482 200 524 238 15.7955

000016 187 187 233 214 15.7955

000016 701 187 756 229 15.7955

000016 794 250 905 371 15.7955

000016 981 184 1098 251 15.7955

000017 253 183 286 206 12.1572

000017 468 166 524 200 12.1572

000017 484 199 535 237 12.1572

000017 112 188 156 209 12.1572

000017 240 182 274 206 12.1572

000017 194 184 244 211 12.1572

000017 709 187 765 228 12.1572

000017 789 245 895 368 12.1572

000017 1001 181 1124 252 12.1572

000018 487 200 538 239 14.984

000018 255 186 291 208 14.984

000018 103 192 156 216 14.984

000018 126 187 156 211 14.984

000018 475 166 535 204 14.984

000018 

In [21]:
with open(output_path, 'w') as output_f:
    for line in unique_records_list:
        output_f.write(line)